In [1]:
import numpy as np

In [48]:
class Tensor:
    """
    Tensor
    """
    def __init__(self, numpy_array):
        if not isinstance(numpy_array, np.ndarray):
            raise ValueError("The class must be initialized with a Numpy array")
        self.value = numpy_array
        self.previous_tensors = None
        self.operation = None  # type of operation done to create this tensor.
        self.grad = np.full(self.value.shape, 1)
        
    def backward(self):
        self.compute_grads(self)
    
    def compute_grads(self, tensor):
        if tensor.previous_tensors is None:
            print('stopped')
#             print('tensor:', tensor)
            return
        else:
            t1, t2 = tensor.previous_tensors
#             print('t1_computed:', str(t1_computed))
#             print('t2_computed:', str(t2_computed))
#             print('t1:', str(t1))
#             print('t2:', str(t2))
#             print('tensor.grad:', str(tensor.grad))
#             if t1_computed is None or t2_computed is None:
#                 return
            if tensor.operation == 'add':
                print('r add')
                t1.grad = tensor.grad
                t2.grad = tensor.grad
            elif tensor.operation == 'mul':
                print('***1')
                print('r mull')
                print('t1.value:', t1.value)
                print('t2.value:', t2.value)
                print('tensor.value', tensor.value)
                print('tensor.grad:', tensor.grad)
                t1.grad = tensor.grad * t2.value # * t1.grad
                t2.grad = tensor.grad * t1.value # * t2.grad
                print('t1.grad:', t1.grad)
                print('t2.grad:', t2.grad)
                print('***2')
            else:
                raise ValueError("Invalid operation")
            self.compute_grads(t1)
            self.compute_grads(t2)
    
            
    def __add__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value + other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'add'
            new_tensor.previous_tensors = [self, other]
            print('add operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
    
    def __mul__(self, other):
        if isinstance(other, Tensor):
            # check shapes
            if not self.value.shape == other.value.shape:
                raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
                                     format(str(self.value.shape), str(other.value.shape)))
            operation_result = self.value * other.value
            new_tensor = Tensor(operation_result)
            new_tensor.operation = 'mul'
            new_tensor.previous_tensors = [self, other]
            print('mul operation happened')
            return new_tensor
        else:
            raise ValueError("Operands must be of `Tensor` type.")
            
            
    
        
#     def __radd__(self, other):
#         return self.__add__(other)
    
#     def __sub__(self, other):
#         if isinstance(other, Tensor):
#             # check shapes
#             if not self.value.shape == other.value.shape:
#                 raise ValueError("Tensors must have the same shape. shape1: {}, shape2: {}".\
#                                      format(str(self.value.shape), str(other.value.shape)))
#             operation_result = self.value - other.value
#             new_tensor = Tensor(operation_result)
#             new_tensor.operation = 'sub'
#             new_tensor.previous_tensors = [self, other]
#             return new_tensor
#         else:
#             raise ValueError("Operands must be of `Tensor` type.")
        
#     def __rsub__(self, other):
#         self.operation = ('rsub', other)
#         operation_result = other - self.value
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
#     def __mul__(self, other):
#         self.operation = ('mul', other)
#         operation_result = self.value * other
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
#     def mean(self):
#         self.operation = ('mean', None)
#         operation_result = self.value.mean()
#         new_tensor = Tensor(numpy_array=operation_result)
#         new_tensor.previous_tensor = self
#         return new_tensor
    
    def __str__(self):
        return self.value.__repr__()

In [49]:
arr1 = np.random.rand(2, 3)
arr2 = np.random.rand(2, 3)

In [50]:
t1 = Tensor(arr1)
t2 = Tensor(arr2)
print('t1:', t1)
print('t2:', t2)

t1: array([[0.09319635, 0.74281403, 0.20479885],
       [0.50613533, 0.6297899 , 0.76707507]])
t2: array([[0.5040132 , 0.45849585, 0.80932387],
       [0.75253264, 0.81481345, 0.75878153]])


In [51]:
t3 = t1 * Tensor(np.full(arr1.shape, 5)) * Tensor(np.full(arr1.shape, 5)) + t2 * Tensor(np.full(arr1.shape, 3))
t4 = (t3 + Tensor(np.full(arr1.shape, 2))) * Tensor(np.full(arr1.shape, 7))

mul operation happened
mul operation happened
mul operation happened
add operation happened
add operation happened
mul operation happened


In [55]:
t4.backward()

***1
r mull
t1.value: [[ 5.84194824 21.94583818  9.54794294]
 [16.91098129 20.18918778 23.45322127]]
t2.value: [[7 7 7]
 [7 7 7]]
tensor.value [[ 40.89363768 153.62086725  66.8356006 ]
 [118.37686904 141.32431445 164.17254886]]
tensor.grad: [[1 1 1]
 [1 1 1]]
t1.grad: [[7 7 7]
 [7 7 7]]
t2.grad: [[ 5.84194824 21.94583818  9.54794294]
 [16.91098129 20.18918778 23.45322127]]
***2
r add
r add
***1
r mull
t1.value: [[0.46598173 3.71407013 1.02399426]
 [2.53067667 3.14894949 3.83537534]]
t2.value: [[5 5 5]
 [5 5 5]]
tensor.value [[ 2.32990863 18.57035063  5.11997132]
 [12.65338337 15.74474743 19.17687668]]
tensor.grad: [[7 7 7]
 [7 7 7]]
t1.grad: [[35 35 35]
 [35 35 35]]
t2.grad: [[ 3.26187209 25.99849088  7.16795985]
 [17.71473672 22.0426464  26.84762735]]
***2
***1
r mull
t1.value: [[0.09319635 0.74281403 0.20479885]
 [0.50613533 0.6297899  0.76707507]]
t2.value: [[5 5 5]
 [5 5 5]]
tensor.value [[0.46598173 3.71407013 1.02399426]
 [2.53067667 3.14894949 3.83537534]]
tensor.grad: [[35 35 3

In [56]:
t1.grad

array([[175, 175, 175],
       [175, 175, 175]])

In [54]:
t2.grad

array([[3, 3, 3],
       [3, 3, 3]])

In [58]:
t3.grad

array([[7, 7, 7],
       [7, 7, 7]])